In [2]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [3]:
# Assume the data is in a directory called 'eye_images'
data_dir = '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/'

# Load the images
X = []
for filename in os.listdir(data_dir):
    if filename.endswith('.tiff'):
        img = Image.open(os.path.join(data_dir, filename))
        img = img.resize((img.width // 2, img.height // 2))  # Example: downsample by 2x
        X.append(np.array(img))
        X.append(np.array(img))
X = np.array(X)

# Normalize the data
X = X / 255.0

In [ ]:


# Define the autoencoder model
"""
model = Sequential()
input_layer = Input(shape=X.shape[1:])

# Define the rest of the model
model.addFlatten()(input_layer)
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))

total_elements = np.prod(X.shape[1:])
model.add(Dense(total_elements, activation='relu')) # Expanding to the total number of elements before reshaping
model.add(Reshape(X.shape[1:]))


model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

"""

input_layer = Input(shape=X.shape[1:])

# Define the rest of the model
x = Flatten()(input_layer)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)
total_elements = np.prod(X.shape[1:])
x = Dense(total_elements, activation='relu')(x) # Expanding to the total number of elements before reshaping

output_layer = Reshape(X.shape[1:])(x)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')


# Train the autoencoder
model.fit(X, X, epochs=100, batch_size=4, shuffle=True, validation_split=0.2)

# Extract the iris color vectors
iris_vectors = model.predict(X)

print(iris_vectors)
del iris_vectors
import gc; gc.collect()
reconstructed_images = model.predict(X)
for i in range(10):
    original_image = X[i]
    reconstructed_image = reconstructed_images[i]
    # Display the original and reconstructed images for comparison
    print(f"Original Image {i + 1}:")
    Image.fromarray((original_image * 255).astype(np.uint8)).show()